In [2]:
import json
import pandas as pd


with open('40344463.json', 'r') as f:
    data = json.load(f)

df = pd.read_csv('44225498.csv')

In [3]:
df.shape

(256963, 18)

In [35]:
idxs = [17 + i * 20 for i in range(10000)]
idx2id = {idx: data['material_id'][str(idx)] for idx in idxs}

In [40]:
from tqdm import tqdm

structs = {idx2id[idx]: data['computed_structure_entry'][str(idx)] for idx in tqdm(idxs)}
energy = {idx2id[idx]: df[df['material_id'] == idx2id[idx]]['e_form_per_atom_wbm'].item() for idx in tqdm(idxs)}
band_gap = {idx2id[idx]: df[df['material_id'] == idx2id[idx]]['bandgap_pbe'].item() for idx in tqdm(idxs)}

  0%|          | 17/10000 [00:00<02:02, 81.40it/s]

100%|██████████| 10000/10000 [02:17<00:00, 72.58it/s]


In [53]:
from pymatgen.io.cif import CifWriter
from pymatgen.core import Structure

In [55]:
res = {
    'cif': [str(CifWriter(Structure.from_dict(structs[idx2id[idx]]['structure']))) for idx in idxs],
    'formation_energy_per_atom': [energy[idx2id[idx]] for idx in idxs],
    'band_gap': [band_gap[idx2id[idx]] for idx in idxs]
}

In [56]:
df1 = pd.DataFrame.from_dict(res)
df1.head()

,cif,formation_energy_per_atom,band_gap
0,# generated using pymatgen\ndata_CeAg2Hg\n_sym...,-0.162,0.0
1,# generated using pymatgen\ndata_InAg2\n_symme...,-0.019,0.0
2,# generated using pymatgen\ndata_SmAg2\n_symme...,-0.289,0.0
3,# generated using pymatgen\ndata_TaAg2\n_symme...,0.281,0.0
4,# generated using pymatgen\ndata_Ag3Bi\n_symme...,0.045,0.0


In [58]:
df1.to_csv('cdvae/data/wbm/val.csv')

Structure Summary
Lattice
    abc : 5.042454641193554 5.0467201298189375 5.046718080465081
 angles : 89.9454568850903 120.03116902810977 120.03404979130008
 volume : 90.7660892380307
      A : 5.04245252 -0.0027327 -0.00373154
      B : -2.52358747 4.37045566 0.00277019
      C : -2.52347539 -1.45416931 4.12150785
    pbc : True True True
PeriodicSite: Ag (-2.524, 1.458, 2.063) [8.43e-06, 0.4999, 0.5001]
PeriodicSite: Ag (2.519, -1.456, 4.117) [1.0, 7.334e-05, 0.9999]
PeriodicSite: Hg (-0.002334, 2.913, 1.031) [0.5, 0.75, 0.25]
PeriodicSite: Ce (-0.002278, 0.0006201, 3.09) [0.5, 0.25, 0.75]

(256963, 18)

-0.178